In [14]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree

In [2]:
train = pd.read_csv('../data/train2.csv', na_values=-1)

In [50]:
train_c0 = train[train['target']==0]
train_c1 = train[train['target']==1]

train_c0 = train_c0.drop('target', axis=1)
train_c1 = train_c1.drop('target', axis=1)

In [157]:
def remove_similar_instances(df, k=3, neigbours=5, leaf_size=2):
    train_c0 = df[df['target']==0]
    train_c1 = df[df['target']==1]
    
    train_c0.fillna(value=999, inplace=True)
    train_c1.fillna(value=999, inplace=True)
    
    features = df.columns.tolist()
    
    tree = KDTree(train_c0.values, leaf_size=leaf_size)
    
    dist, ind = tree.query(train_c1.values, k=k) 
    X_new = np.delete(train_c0.values, (ind[dist < neigbours]), axis=0)
    X_deleted = train_c0.values[ind[dist < neigbours],:]
    
    X_easy = np.vstack((train_c1.values, X_new))
    X_hard = np.vstack((train_c1.values, X_deleted))
    
    df_easy = pd.DataFrame(data=X_easy, columns=features)
    df_hard = pd.DataFrame(data=X_hard, columns=features)
    
    return df_easy, df_hard


In [159]:
df_easy, df_hard = remove_similar_instances(train, k=3, neigbours=5, leaf_size=2)

/home/laith/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [165]:
df_hard.head()

,ps_car_13,ps_reg_03,ps_ind_05_cat,ps_ind_03,ps_ind_15,ps_reg_02,ps_car_14,ps_car_12,ps_car_01_cat,ps_car_07_cat,...,ps_car_04_cat_avg,ps_car_09_cat_avg,ps_car_02_cat_avg,ps_ind_02_cat_avg,ps_car_05_cat_avg,ps_car_08_cat_avg,ps_ind_04_cat_avg,ps_reg_01_plus_ps_car_02_cat_avg,ps_reg_01_plus_ps_car_04_cat_avg,target
0,1.458184,2.316652,0.0,2.0,4.0,1.4,0.390256,0.447214,11.0,1.0,...,0.048825,0.036319,0.049507,0.035725,0.040111,0.034735,0.034790,0.051886,0.049422,1.0
1,0.464593,0.402337,0.0,2.0,8.0,0.3,0.361248,0.315278,7.0,1.0,...,0.033451,0.036319,0.033772,0.035725,0.036448,0.034735,0.038644,0.038214,0.038347,1.0
2,0.720265,999.000000,4.0,0.0,8.0,0.3,0.441588,0.447214,4.0,1.0,...,0.033451,0.033344,0.049507,0.038139,0.040454,0.034735,0.038644,0.042417,0.024401,1.0
3,0.740165,0.920937,0.0,3.0,12.0,0.0,0.359861,0.316228,6.0,1.0,...,0.033451,0.033344,0.033772,0.035725,0.036448,0.034735,0.038644,0.029019,0.027963,1.0
4,1.023898,0.812404,0.0,1.0,8.0,0.3,0.408656,0.387298,11.0,1.0,...,0.033451,0.036319,0.049507,0.038139,0.036448,0.034735,0.034790,0.048988,0.036098,1.0


In [166]:
df_hard.to_csv('../data/train-hard.csv', index=False)
df_easy.to_csv('../data/train-easy.csv', index=False)